# Cointegration Tests & Pairs Trading

### Loading Libraries

In [9]:
# Numerical Computing
import numpy as np
from numpy.linalg import LinAlgError

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms

# StatsModel
import statsmodels.tsa.api as tsa
from statsmodels.tsa.api import VAR
from statsmodels.tsa.api import VARMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller, coint

# Arch
from arch import arch_model
from arch.univariate import ConstantMean, GARCH, Normal

# Scipy
from scipy.stats import probplot, moment

# Warnings
import warnings

# Notebook Optimizer
from tqdm import tqdm

# Itertools
from itertools import product

# Path & Notebook Optimizer
from tqdm import tqdm 
from pathlib import Path


# Scikit-Learn
from sklearn.preprocessing import minmax_scale
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix

In [10]:
%matplotlib inline

In [11]:
sns.set(style='whitegrid',
        context='notebook',
        color_codes=True)

warnings.filterwarnings('ignore')

In [12]:
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

In [14]:
DATA_PATH = Path('..', 'data')

STORE = DATA_PATH / 'assets.h5'

### Johansen Test Critical Values

In [15]:
critical_values = {0: {.9: 13.4294, .95: 15.4943, .99: 19.9349},
                   1: {.9: 2.7055, .95: 3.8415, .99: 6.6349}}

In [ ]:
# Critical Value for 0 Cointegration Relationships
trace0_cv = critical_values[0][.95] 

# Critical Value for 1 Cointegration Relationship
trace1_cv = critical_values[1][.95] 

### Load & Clean Stock & ETF Data

#### Remove Highly Correlated Assets

In [16]:
def remove_correlated_assets(df, cutoff=.99):
    corr = df.corr().stack()
    corr = corr[corr < 1]
    to_check = corr[corr.abs() > cutoff].index
    keep, drop = set(), set()
    for s1, s2 in to_check:
        if s1 not in keep:
            if s2 not in keep:
                keep.add(s1)
                drop.add(s2)
            else:
                drop.add(s1)
        else:
            keep.discard(s2)
            drop.add(s2)
    return df.drop(drop, axis=1)

#### Remove Stationary Series

In [17]:
def check_stationarity(df):
    results = []
    for ticker, prices in df.items():
        results.append([ticker, adfuller(prices, regression='ct')[1]])
    return pd.DataFrame(results, columns=['ticker', 'adf']).sort_values('adf')

In [18]:
def remove_stationary_assets(df, pval=.05):
    test_result = check_stationarity(df)
    stationary = test_result.loc[test_result.adf <= pval, 'ticker'].tolist()
    return df.drop(stationary, axis=1).sort_index()

#### Assets Selection

In [19]:
def select_assets(asset_class='stocks', n=500, start=2010, end=2019):
    idx = pd.IndexSlice
    with pd.HDFStore(STORE) as store:
        df = (pd.concat([store[f'stooq/us/nasdaq/{asset_class}/prices'],
                         store[f'stooq/us/nyse/{asset_class}/prices']])
              # stooq download can have duplicate assets
              .loc[lambda df: ~df.index.duplicated()]
              .sort_index()
              .loc[idx[:, f'{start}':f'{end}'], :]
              .assign(dv=lambda df: df.close.mul(df.volume)))

    # select n assets with the highest average trading volume
    # we are taking a shortcut to simplify; should select
    # based on historical only, e.g. yearly rolling avg
    most_traded = (df.groupby(level='ticker')
                   .dv.mean()
                   .nlargest(n=n).index)

    df = (df.loc[idx[most_traded, :], 'close']
          .unstack('ticker')
          .ffill(limit=5)  
          .dropna(axis=1))  

    df = remove_correlated_assets(df)
    return remove_stationary_assets(df).sort_index()

In [21]:
for asset_class, n in [('etfs', 500), ('stocks', 250)]:
    df = select_assets(asset_class=asset_class, n=n)
    df.to_hdf('data.h5', f'{asset_class}/close')

#### Getting Ticker Dictionary

In [22]:
def get_ticker_dict():
    with pd.HDFStore(STORE) as store:
        return (pd.concat([
            store['stooq/us/nyse/stocks/tickers'],
            store['stooq/us/nyse/etfs/tickers'],
            store['stooq/us/nasdaq/etfs/tickers'],
            store['stooq/us/nasdaq/stocks/tickers']
        ]).drop_duplicates().set_index('ticker').squeeze().to_dict())

In [24]:
names = get_ticker_dict()

### Visualize Correlation Clusters

In [25]:
stocks = pd.read_hdf('data.h5', 'stocks/close')
stocks.info()

In [26]:
etfs = pd.read_hdf('data.h5', 'etfs/close')
etfs.info()

In [27]:
tickers = {k: v for k, v in names.items() if k in etfs.columns.union(stocks.columns)}
pd.Series(tickers).to_hdf('data.h5', 'tickers')

In [28]:
corr = pd.DataFrame(index=stocks.columns)

for etf, data in etfs.items():
    corr[etf] = stocks.corrwith(data)

In [29]:
corr.info()

In [30]:
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.clustermap(corr, cmap=cmap, center=0);

plt.show()

## Candidate Selection using Heuristics

### Computational Complexity: Comparing Running Times

#### Preparing Data

In [31]:
stocks.shape, etfs.shape

In [32]:
stocks.info()

In [33]:
etfs.info()

In [35]:
security = etfs['AAXJ.US'].loc['2010': '2013']

candidates = stocks.loc['2010': '2013']

In [36]:
security = security.div(security.iloc[0])

candidates = candidates.div(candidates.iloc[0])

spreads = candidates.sub(security, axis=0)

In [40]:
n, m = spreads.shape

X = np.ones(shape=(n, 2))

X[:, 1] = np.arange(1, n+1)

#### Heuristics

In [41]:
%%timeit
np.linalg.inv(X.T @ X) @ X.T @ spreads

In [42]:
%%timeit
spreads.std()

In [43]:
%%timeit
candidates.corrwith(security)

#### Cointegration Tests

In [44]:
# %%timeit
# for candidate, prices in candidates.items():
#     df = pd.DataFrame({'s1': security,
#                        's2': prices})
#     var = VAR(df.values)
#     lags = var.select_order()
#     k_ar_diff = lags.selected_orders['aic']
#     coint_johansen(df, det_order=0, k_ar_diff=k_ar_diff)
#     coint(security, prices, trend='c')[:2]
#     coint(prices, security, trend='c')[:2]

### Computing Heuristics

In [45]:
def compute_pair_metrics(security, candidates):
    security = security.div(security.iloc[0])
    ticker = security.name
    candidates = candidates.div(candidates.iloc[0])
    spreads = candidates.sub(security, axis=0)
    n, m = spreads.shape
    X = np.ones(shape=(n, 2))
    X[:, 1] = np.arange(1, n + 1)
    
    # compute drift
    drift = ((np.linalg.inv(X.T @ X) @ X.T @ spreads).iloc[1]
             .to_frame('drift'))
    
    # compute volatility
    vol = spreads.std().to_frame('vol')
    
    # return correlation
    corr_ret = (candidates.pct_change()
                .corrwith(security.pct_change())
                .to_frame('corr_ret'))
    
    # normalized price series correlation
    corr = candidates.corrwith(security).to_frame('corr')
    metrics = drift.join(vol).join(corr).join(corr_ret).assign(n=n)
    
    tests = []
    # run cointegration tests
    for candidate, prices in tqdm(candidates.items()):
        df = pd.DataFrame({'s1': security, 's2': prices})
        var = VAR(df.values)
        lags = var.select_order() # select VAR order
        k_ar_diff = lags.selected_orders['aic']
        # Johansen Test with constant Term and estd. lag order
        cj0 = coint_johansen(df, det_order=0, k_ar_diff=k_ar_diff)
        # Engle-Granger Tests
        t1, p1 = coint(security, prices, trend='c')[:2]
        t2, p2 = coint(prices, security, trend='c')[:2]
        tests.append([ticker, candidate, t1, p1, t2, p2, 
                      k_ar_diff, *cj0.lr1])
    columns = ['s1', 's2', 't1', 'p1', 't2', 'p2', 'k_ar_diff', 'trace0', 'trace1']
    tests = pd.DataFrame(tests, columns=columns).set_index('s2')
    return metrics.join(tests)

In [46]:
spreads = []

start = 2010
stop = 2019

etf_candidates = etfs.loc[str(start): str(stop), :]
stock_candidates = stocks.loc[str(start): str(stop), :]
s = time()

for i, (etf_ticker, etf_prices) in enumerate(etf_candidates.items(), 1):
    df = compute_pair_metrics(etf_prices, stock_candidates)
    spreads.append(df.set_index('s1', append=True))
    if i % 10 == 0:
        print(f'\n{i:>3} {time() - s:.1f}\n')
        s = time()

In [47]:
names = get_ticker_dict()

spreads = pd.concat(spreads)
spreads.index.names = ['s2', 's1']
spreads = spreads.swaplevel()

spreads['name1'] = spreads.index.get_level_values('s1').map(names)
spreads['name2'] = spreads.index.get_level_values('s2').map(names)

In [48]:
spreads['t'] = spreads[['t1', 't2']].min(axis=1)

spreads['p'] = spreads[['p1', 'p2']].min(axis=1)

### Engle-Granger vs Johansen: How Do Their Findings Compare?

In [ ]:
spreads['trace_sig'] = ((spreads.trace0 > trace0_cv) &
                        (spreads.trace1 > trace1_cv)).astype(int)

spreads['eg_sig'] = (spreads.p < .05).astype(int)

In [49]:
pd.crosstab(spreads.eg_sig, spreads.trace_sig)

In [50]:
spreads['coint'] = (spreads.trace_sig & spreads.eg_sig).astype(int)

In [51]:
spreads.info()

In [52]:
spreads = spreads.reset_index()

In [53]:
sns.scatterplot(x=np.log1p(spreads.t.abs()), 
                y=np.log1p(spreads.trace1), 
                hue='coint', data=spreads[spreads.trace0>trace0_cv]);
plt.show()

In [54]:
spreads.to_hdf('heuristics.h5', 'spreads')

In [55]:
spreads = pd.read_hdf('heuristics.h5', 'spreads')

#### Evaluate Heuristics

In [56]:
spreads.drift = spreads.drift.abs()

In [57]:
pd.crosstab(spreads.eg_sig, spreads.trace_sig)

In [58]:
pd.set_option('display.float_format', lambda x: f'{x:.2%}')

pd.crosstab(spreads.eg_sig, spreads.trace_sig, normalize=True)

In [59]:
fig, axes = plt.subplots(ncols=4, figsize=(20, 5))

for i, heuristic in enumerate(['drift', 'vol', 'corr', 'corr_ret']):
    sns.boxplot(x='coint', y=heuristic, data=spreads, ax=axes[i])

fig.tight_layout();
plt.show()

## How Well Do The Heuristics Predict Significant Cointegration?

In [60]:
spreads.groupby(spreads.coint)['drift', 'vol', 'corr'].describe().stack(level=0).swaplevel().sort_index()

In [61]:
spreads.coint.value_counts()

### Logistic Regression

In [62]:
y = spreads.coint

X = spreads[['drift', 'vol', 'corr', 'corr_ret']]

# X = spreads[['drift', 'vol']]

In [63]:
kf = StratifiedKFold(n_splits=5, shuffle=True)

In [64]:
log_reg = LogisticRegressionCV(Cs=np.logspace(-10, 10, 21), 
                               class_weight='balanced',
                               scoring='roc_auc')

In [65]:
log_reg.fit(X=X, y=y)

Cs = log_reg.Cs_

scores = pd.DataFrame(log_reg.scores_[True], columns=Cs).mean()
scores.plot(logx=True);

f'C:{np.log10(scores.idxmax()):.2f}, AUC: {scores.max():.2%}'

In [66]:
log_reg.coef_

In [67]:
y_pred = log_reg.predict_proba(X)[:, 1]

confusion_matrix(y_true=spreads.coint, y_pred=(y_pred>.5))

In [68]:
spreads.assign(y_pred=log_reg.predict_proba(X)[:, 1]).groupby(spreads.coint).y_pred.describe()

### Decision Tree Classifier

In [69]:
model = DecisionTreeClassifier(class_weight='balanced')

decision_tree = GridSearchCV(model,
                             param_grid={'max_depth': list(range(1, 10))},
                             cv=5,
                             scoring='roc_auc')

In [70]:
decision_tree.fit(X=X, y=y)

In [71]:
f'{decision_tree.best_score_:.2%}, Depth: {decision_tree.best_params_["max_depth"]}'

In [72]:
pd.Series(data=decision_tree.best_estimator_.feature_importances_, 
          index=X.columns).sort_values().plot.barh(title='Feature Importance')

sns.despine();
plt.show()

In [74]:
spreads.assign(y_pred=decision_tree.predict_proba(X)[:, 1]).groupby(spreads.coint).y_pred.describe()

In [73]:
sns.catplot(x='coint', 
            y='y_pred', 
            data=spreads.assign(y_pred=decision_tree.predict_proba(X)[:, 1]), 
            kind='box');

plt.show()